In [159]:
from ltlcross_runner import LtlcrossRunner
from IPython.display import display
import pandas as pd
import spot
import sys
import re
spot.setup(show_default='.a')
pd.options.display.float_format = '{: .0f}'.format
pd.options.display.latex.multicolumn_format = 'c'
from tables_utils import split_cols, high_min, high_max, highlight_by_level
from tables_utils import fix_latex, fix_type, fix_tool
from tables_utils import cummulative_to_latex

With `rerun = True`, all experiments are executed again (takes several hours). With `False`, the data are taken from the *.csv files:

In [177]:
rerun = False

In [161]:
import os
os.environ['SPOT_HOA_TOLERANT']='TRUE'

In [192]:
%%bash
ltl3ba -v
ltl3tela -v
ltlcross --version

LTL3BA 1.1.3
LTL3TELA 2.1.0 (using Spot 2.7.4)
ltlcross (spot) 2.7.4

Copyright (C) 2019  Laboratoire de Recherche et Développement de l'Epita.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.


The `generate` function may be used to generate random formulae. Uncomment the function call below to generate different set of formulae.

In [193]:
def generate(n=1000,func=(lambda x: True),filename=None,priorities='',ap=['a','b','c','d','e']):
    if filename is None:
        file_h = sys.stdout
    else:
        file_h = open(filename,'w')
    f = spot.randltl(ap,
                     ltl_priorities=priorities,
                     simplify=3,tree_size=15).relabel_bse(spot.Abc)
    i = 0
    printed = set()
    while(i < n):
        form = next(f)
        if form in printed:
            continue
        if func(form) and not form.is_tt() and not form.is_ff():
            print(form,file=file_h)
            printed.add(form)
            i += 1

In [194]:
datasets = [ {
    'name': 'rand1',
    'form': 'formulae/ictac19/rand1.ltl',
    'csv' : 'formulae/ictac19/rand1.csv',
    'dcsv': 'formulae/ictac19/rand1.d.csv',
    'ncsv': 'formulae/ictac19/rand1.n.csv',
}, {
    'name': 'rand2',
    'form': 'formulae/ictac19/rand2.ltl',
    'csv' : 'formulae/ictac19/rand2.csv',
    'dcsv': 'formulae/ictac19/rand2.d.csv',
    'ncsv': 'formulae/ictac19/rand2.n.csv',
}, {
    'name': 'rand4',
    'form': 'formulae/ictac19/rand4.ltl',
    'csv' : 'formulae/ictac19/rand4.csv',
    'dcsv': 'formulae/ictac19/rand4.d.csv',
    'ncsv': 'formulae/ictac19/rand4.n.csv',
}, {
    'name': 'randfg',
    'form': 'formulae/ictac19/randfg.ltl',
    'csv' : 'formulae/ictac19/randfg.csv',
    'dcsv': 'formulae/ictac19/randfg.d.csv',
    'ncsv': 'formulae/ictac19/randfg.n.csv',
}, {
    'name': 'literature',
    'form': 'formulae/ictac19/literature.ltl',
    'csv' : 'formulae/ictac19/literature.csv',
    'dcsv': 'formulae/ictac19/literature.d.csv',
    'ncsv': 'formulae/ictac19/literature.n.csv',
} ]

In [195]:
tools = {
    "ltl3ba": "ltldo 'ltl3ba -H1' -f %f > %O",
    "basic": "ltl3tela -p1 -F0 -G0 -i1 -X1 -f %f > %O",
    "fmerg": "ltl3tela -p1 -G0 -i1 -X1 -f %f > %O",
    "fgmerg": "ltl3tela -p1 -i1 -X1 -f %f > %O",
}
order = ["ltl3ba", "basic", "fmerg", "fgmerg"]
cols = ["states", "edges", "acc"]
cols_ = cols + ["nonalt", "det"] # we will count these fields later

In [196]:
for d in datasets:
    d['data'] = LtlcrossRunner(tools, formula_files = [d['form']], res_filename = d['csv'], cols = cols)
    if rerun:
        d['data'].run_ltlcross(automata = True, timeout = '60')
    d['data'].parse_results()

For all datasets, let us count the number of existential (i.e. automata without universal branching) and deterministic automata produced by the translation to alternating automaton. 

In [197]:
for d in datasets:
    for tool in tools:
        d['data'].values[('nonalt', tool)] = list(map(lambda aut : \
            1 if type(aut) == str and spot.automaton(aut + '\n').is_existential() else 0, d['data'].automata[tool]))
    for tool in tools:
        d['data'].values[('det', tool)] = list(map(lambda aut: \
            1 if type(aut) == str and spot.automaton(aut + '\n').is_deterministic() else 0, d['data'].automata[tool]))

In [198]:
for d in datasets:
    if d['name'] == 'literature':
        # we filter formulae to those where FG-merging helped compared to LTL3BA
        litdf = d['data'].values
        d['data'].values = litdf[litdf[('states', 'fgmerg')] < litdf[('states', 'basic')]]
        print("Number of mergeable formulae from literature: {}".format(len(d['data'].values)))
    d['dc'] = d['data'].cummulative(col = cols_).unstack(level = 0).loc[order, cols_]

Number of mergeable formulae from literature: 24


In [199]:
m = datasets[0]['dc'].copy()
m.columns = m.columns.map(lambda c : c + '.' + datasets[0]['name'])
for i in range(1, len(datasets)):
    ds = datasets[i]['dc'].copy()
    ds.columns = ds.columns.map(lambda c : c + '.' + datasets[i]['name'])
    m = pd.merge(m, ds, on = 'tool')

In [200]:
t1 = split_cols(m, '.').loc[order, ['states', 'acc']].sort_index(1, ascending = [False, True])

In [201]:
t2 = split_cols(m, '.').loc[order, ['det', 'nonalt']].sort_index(1)

In [202]:
t1

states                                 acc                         
       literature rand1 rand2 rand4 randfg literature rand1 rand2 rand4 randfg
tool                                                                          
ltl3ba        140  6253  6313  6412   5051         24  1000  1000  1000   1000
basic         140  6234  6287  6393   5051         24   997  1000  1000   1000
fmerg         110  5418  5296  5231   3926         46  1160  1244  1347   1343
fgmerg         65  4595  4300  4015   2744         98  2971  3317  3677   2978

In [203]:
t2

det                              nonalt                         
       literature rand1 rand2 rand4 randfg literature rand1 rand2 rand4 randfg
tool                                                                          
ltl3ba          0     5     2     0      0          6   148   114    89    144
basic           0     5     2     0      0          6   148   114    89    144
fmerg           2    64    53    40     73          6   171   146   119    192
fgmerg         10   133   126   124    217         18   356   367   385    603

In [204]:
print(t1.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{c}{states} & \multicolumn{5}{c}{acc} \\
{} & literature & rand1 & rand2 & rand4 & randfg & literature & rand1 & rand2 & rand4 & randfg \\
tool   &            &       &       &       &        &            &       &       &       &        \\
\midrule
ltl3ba &        140 &  6253 &  6313 &  6412 &   5051 &         24 &  1000 &  1000 &  1000 &   1000 \\
basic  &        140 &  6234 &  6287 &  6393 &   5051 &         24 &   997 &  1000 &  1000 &   1000 \\
fmerg  &        110 &  5418 &  5296 &  5231 &   3926 &         46 &  1160 &  1244 &  1347 &   1343 \\
fgmerg &         65 &  4595 &  4300 &  4015 &   2744 &         98 &  2971 &  3317 &  3677 &   2978 \\
\bottomrule
\end{tabular}



In [205]:
print(t2.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{c}{det} & \multicolumn{5}{c}{nonalt} \\
{} & literature & rand1 & rand2 & rand4 & randfg & literature & rand1 & rand2 & rand4 & randfg \\
tool   &            &       &       &       &        &            &       &       &       &        \\
\midrule
ltl3ba &          0 &     5 &     2 &     0 &      0 &          6 &   148 &   114 &    89 &    144 \\
basic  &          0 &     5 &     2 &     0 &      0 &          6 &   148 &   114 &    89 &    144 \\
fmerg  &          2 &    64 &    53 &    40 &     73 &          6 &   171 &   146 &   119 &    192 \\
fgmerg &         10 &   133 &   126 &   124 &    217 &         18 &   356 &   367 &   385 &    603 \\
\bottomrule
\end{tabular}



### Scatter plots

In [206]:
def fix_tools(tool):
    return tool.replace('FG-','$\\FG$-').replace('F-','$\\F$-')

In [207]:
def sc_plot(r,t1,t2,filename=None,include_equal = True,col='states',log=None,size=(5.5,5),kw=None,clip=None, add_count=True):
    merged = isinstance(r,list)
    if merged:
        vals = pd.concat([run.values[col] for run in r])
        vals.index = vals.index.droplevel(0)
        vals = vals.groupby(vals.index).first()
    else:
        vals = r.values[col]
    to_plot = vals.loc(axis=1)[[t1,t2]] if include_equal else\
        vals[vals[t1] != vals[t2]].loc(axis=1)[[t1,t2]]
    to_plot['count'] = 1
    to_plot.dropna(inplace=True)
    to_plot = to_plot.groupby([t1,t2]).count().reset_index()
    if filename is not None:
        print(scatter_plot(to_plot, log=log, size=size,kw=kw,clip=clip, add_count=add_count),file=open(filename,'w'))
    else:
        return scatter_plot(to_plot, log=log, size=size,kw=kw,clip=clip, add_count=add_count)

In [208]:
def scatter_plot(df, short_toolnames=True, log=None, size=(5.5,5),kw=None,clip=None,add_count = True):
    t1, t2, _ = df.columns.values
    if short_toolnames:
        t1 = fix_tools(t1.split('/')[0])
        t2 = fix_tools(t2.split('/')[0])
    vals = ['({},{}) [{}]\n'.format(v1,v2,c) for v1,v2,c in df.values]
    plots = '''\\addplot[
    scatter, scatter src=explicit, 
    only marks, fill opacity=0.5,
    draw opacity=0] coordinates
    {{{}}};'''.format(' '.join(vals))
    start_line = 0 if log is None else 1
    line = '\\addplot[darkgreen,domain={}:{}]{{x}};'.format(start_line, min(df.max(axis=0)[:2])+1)
    axis = 'axis'
    mins = 'xmin=0,ymin=0,'
    clip_str = ''
    if clip is not None:
        clip_str = '\\draw[red,thick] ({},{}) rectangle ({},{});'.format(*clip)
    if log:
        if log == 'both':
            axis = 'loglogaxis'
            mins = 'xmin=1,ymin=1,'
        else:
            axis = 'semilog{}axis'.format(log)
            mins = mins + '{}min=1,'.format(log)
    args = ''
    if kw is not None:
        if 'title' in kw and add_count:
            kw['title'] = '{{{} ({})}}'.format(kw['title'],df['count'].sum())
        args = ['{}={},\n'.format(k,v) for k,v in kw.items()]
        args = ''.join(args)
    res = '''%\\begin{{tikzpicture}}
\\pgfplotsset{{every axis legend/.append style={{
cells={{anchor=west}},
draw=none,
}}}}
\\pgfplotsset{{colorbar/width=.3cm}}
\\pgfplotsset{{title style={{align=center,
                        font=\\small}}}}
\\pgfplotsset{{compat=1.14}}
\\begin{{{0}}}[
{1}
colorbar,
colormap={{example}}{{
  color(0)=(blue)
  color(500)=(green)
  color(1000)=(red)
}},
%thick,
axis x line* = bottom,
axis y line* = left,
width={2}cm, height={3}cm, 
xlabel={{{4}}},
ylabel={{{5}}},
cycle list={{%
{{darkgreen, solid}},
{{blue, densely dashed}},
{{red, dashdotdotted}},
{{brown, densely dotted}},
{{black, loosely dashdotted}}
}},
{6}%
]
{7}%
{8}%
{9}%
\\end{{{0}}}
%\\end{{tikzpicture}}
'''.format(axis,mins,
                    size[0],size[1],t1,t2,
                    args,plots,line,
                    clip_str)
    return res

In [214]:
fgm = 'fgmerg'
basic = 'basic'

size = (4,4)
clip_names = ('xmin','ymin','xmax','ymax')
kw = {}
sc_plot(datasets[4]['data'],basic,fgm,'sc_lit.tex',size=size,kw=kw.copy())

size = (4.3,4.5)
kw['title'] = 'literature'
sc_plot(datasets[4]['data'],basic,fgm,'sc_lit.tex',size=size,kw=kw.copy())
i = 0
for suff in ['1','2','4','fg']:
    kw['title'] = 'rand'+suff
    sc_plot(datasets[i]['data'],basic,fgm,'sc_rand{}.tex'.format(suff),size=size,kw=kw.copy())
    i += 1